In [1]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'

fairing.config.set_preprocessor(
    'python', 
     # command = ['python'],  # default: python
     input_files = ['00-python-file-to-fairing.py'],     
)

fairing.config.set_builder(
    'append',
    base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image
    registry = PRIVATE_REGISTRY,
    image_name='my-02-python-file-fairing', 
    push=True
)

fairing.config.set_deployer(
    'job',
    namespace='myspace',
    pod_spec_mutators=[
        k8s_utils.get_resource_mutator(cpu=1, memory=5)]
)

fairing.config.run()

[I 201207 13:41:50 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7fd1a05ef4a8>
[I 201207 13:41:50 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fd1a05ef3c8>
[I 201207 13:41:50 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7fd1a05ef470>
[W 201207 13:41:50 append:50] Building image using Append builder...
[I 201207 13:41:50 base:107] Creating docker context: /tmp/fairing_context_ofezrkfc
[I 201207 13:41:50 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/kf-base:latest'
[W 201207 13:41:50 append:54] Image successfully built in 0.029696974999751546s.
[W 201207 13:41:50 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/my-02-python-file-fairing:E2683407...
[I 201207 13:41:50 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.clust

2020-12-07 13:42:20.387742: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-12-07 13:42:20.387846: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-12-07 13:42:20.387865: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
11493376/11490434 [==============================] - 0s 0us/step
2020-12-07 13:42:21.855682: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or direct

[W 201207 13:42:52 job:173] Cleaning up job fairing-job-29mnl...


(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7fd1a05ef4a8>,
 <kubeflow.fairing.deployers.job.job.Job at 0x7fd1a05ef470>)

# Image 확인
- Host에서 실행
- fairing에서 난수로 생성한 tag를 변경해서 실행할 것 (예: E2683407)

### Image pull
```sh
docker pull kubeflow-registry.default.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```

### Image inspect
```sh
docker inspect kubeflow-registry.default.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```
Cmd 항목을 확인하고 faring 코드대로 image build 되었음을 확인
```json
...
            "Cmd": [
                "python",
                "/app/00-python-file-to-fairing.py"
            ],
...
```